In [ ]:
import os
from os import sys
sys.path.append('../../MpowerAI')
from pympower.classes.mshaai import MShaAI

shaai = MShaAI()

In [ ]:
srcPath = "./data/경조사지원규정.pdf"
tgtName = "경조사지원규정.txt"
OUT_FOLDER = "./data/out"

if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

tgtPath = f"{OUT_FOLDER}/{tgtName}"

shaai.extract(srcPath=srcPath, tgtPath=tgtPath)

In [ ]:
# URL 이미지 => 저장 하는 예제
import requests
from io import BytesIO
from PIL import Image

def download_save(url:str, max_size=(128, 128)):
    save_path = "./data/test.jpg"
    
    #이미지다운로드
    response = requests.get(url)
    if response.status_code == 200:
        # 이미지 열기
        img = Image.open(BytesIO(response.content))
      
        # [bong][2024-06-28] 
        # 노래만들기 할때 .PNG 파일 업로드 하면 에러남.
        # => PNG 파일은 투명도를 포함할 수 있는 RGBA 모드를 사용할 수 있는데, 그러나 JPEG는 투명도를 지원하지 않기 때문에 에러남.
        #=> 따라서 PNG 파일을 JPEG로 변환시 투명 부분을 흰색으로 처리함.
        if img.mode == 'RGBA':
            # 흰색 배경의 새로운 이미지 생성
            background = Image.new("RGB", img.size, (255, 255, 255))
            # 기존 이미지를 배경 이미지에 덮어쓰기
            background.paste(img, (0, 0), img)
            img = background
            
        # 이미지 크기 확인 및 조정
        if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
            img.thumbnail(max_size)
            
        # JPG 형식으로 저장
        img.save(save_path, "JPEG")
        print(f"이미지가 성공적으로 저장되었습니다: {save_path}")
 
url = "https://talk.kakaocdn.net/dna/cgCWKI/bl6V8v6z9s5/rQjDUMvgkLE9Nfv9vbdUPj/i_c75fc38092bd.png?credential=zf3biCPbmWRjbqf40YGePFLewdou7TIK&expires=1814139966&signature=PqVn8TnouSuPJzXFqWzWZRxBZqI%3D"

download_save(url=url)

In [ ]:
from utils import MyUtils
myutils = MyUtils(yam_file_path='./data/settings.yaml')

OUT_FOLDER = "./files/out"
file_paths = myutils.getListOfFiles(OUT_FOLDER)
for idx, file_path in enumerate(file_paths):
    if '.ipynb_checkpoints' not in file_path:
        print(file_path)
            